In [1]:
import pandas as pd

# Load the datasets
customers_df = pd.read_csv('Customers.csv')
transactions_df = pd.read_csv('Transactions.csv')
products_df = pd.read_csv('Products.csv')

print("Data shapes:")
print("Customers:", customers_df.shape)
print("Transactions:", transactions_df.shape)
print("Products:", products_df.shape)

print("\
Sample of transactions:")
print(transactions_df.head())

Data shapes:
Customers: (200, 4)
Transactions: (1000, 7)
Products: (100, 4)
Sample of transactions:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue   Price  
0      300.68  300.68  
1      300.68  300.68  
2      300.68  300.68  
3      601.36  300.68  
4      902.04  300.68  


In [3]:
# Check the columns of the merged data to ensure 'Price' exists
merged_data = transactions_df.merge(customers_df, on='CustomerID')
merged_data = merged_data.merge(products_df, on='ProductID')

print("Columns in merged data:")
print(merged_data.columns)

# Display a sample of the merged data to verify
print("\
Sample of merged data:")
print(merged_data.head())

Columns in merged data:
Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')
Sample of merged data:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  

In [4]:
# Inspect the 'Price_x' and 'Price_y' columns to determine their values
print("Unique values in Price_x:")
print(merged_data['Price_x'].unique())

print("\
Unique values in Price_y:")
print(merged_data['Price_y'].unique())

Unique values in Price_x:
[300.68 217.05 239.7  147.95 290.17 274.94 454.53 100.81 227.26 374.16
 456.28 155.65  57.3   48.69 350.13 153.19 420.15 147.22 416.35 156.96
 337.91  44.12  65.16 440.7  159.33 447.23 389.2  429.29 372.55 114.2
 157.62 437.65 277.86 284.62 229.06  41.18 126.34 430.59 330.05 137.54
 335.84 399.34 303.2  196.4  379.44 295.58 488.63 325.01  66.32 261.2
 307.47 469.77 416.4  354.81 427.61  33.59 220.91 263.55 337.22 158.93
 169.3  304.94  26.99 121.32 128.53 452.42 165.3  436.89 455.72 404.4
  30.59  64.25  26.26  95.69 433.64 319.34 303.5  354.29 385.37 396.34
  18.82 346.3   16.08  55.99 159.6  497.76 429.31 235.58 447.34 417.37
 338.66 146.85 222.95 127.36 299.93  64.38 481.78 366.07 265.51 459.86]
Unique values in Price_y:
[300.68 217.05 239.7  147.95 290.17 274.94 454.53 100.81 227.26 374.16
 456.28 155.65  57.3   48.69 350.13 153.19 420.15 147.22 416.35 156.96
 337.91  44.12  65.16 440.7  159.33 447.23 389.2  429.29 372.55 114.2
 157.62 437.65 277.86 284.62

In [5]:
# Build the lookalike model with corrected features
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np

# Prepare customer profiles
customer_profile = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Price_x': 'mean',
    'Region': 'first',
    'Category': lambda x: ','.join(set(x))
}).reset_index()

# One-hot encode categorical variables
customer_profile = pd.get_dummies(customer_profile, columns=['Region'])

# Scale numerical features
scaler = StandardScaler()
numerical_features = ['TotalValue', 'Quantity', 'Price_x']
customer_profile[numerical_features] = scaler.fit_transform(customer_profile[numerical_features])

# Compute similarity matrix
similarity_matrix = cosine_similarity(customer_profile.drop(columns=['CustomerID', 'Category']))

# Find top 3 similar customers for first 20 customers
lookalike_results = []
for i in range(20):
    customer_id = customer_profile['CustomerID'].iloc[i]
    similar_indices = np.argsort(-similarity_matrix[i])[1:4]
    similar_customers = customer_profile.iloc[similar_indices]['CustomerID'].values
    similarity_scores = similarity_matrix[i][similar_indices]

    for similar_customer, score in zip(similar_customers, similarity_scores):
        lookalike_results.append({
            'CustomerID': customer_id,
            'SimilarCustomerID': similar_customer,
            'SimilarityScore': round(score, 4)
        })

# Create and save results
lookalike_df = pd.DataFrame(lookalike_results)
lookalike_df.to_csv('Lookalike.csv', index=False)

print("Top 3 lookalikes for first 20 customers:")
print(lookalike_df.head(10))

Top 3 lookalikes for first 20 customers:
  CustomerID SimilarCustomerID  SimilarityScore
0      C0001             C0137           0.9790
1      C0001             C0191           0.9706
2      C0001             C0011           0.9489
3      C0002             C0088           0.9848
4      C0002             C0142           0.9783
5      C0002             C0043           0.9638
6      C0003             C0190           0.9807
7      C0003             C0147           0.9558
8      C0003             C0174           0.9319
9      C0004             C0165           0.9826
